# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Create a Keyspace 
create_keyspace_query = '''
CREATE KEYSPACE IF NOT EXISTS temp_project
    WITH REPLICATION =  {
        'class' : 'SimpleStrategy', 
        'replication_factor' : 1
    }
'''

try:
    session.execute(create_keyspace_query)
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('temp_project')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Drop the table if it exists
query_drop_table_1 = "DROP TABLE IF EXISTS music_app_history"
try:
    session.execute(query_drop_table_1)
except Exception as e:
    print(e)

# Create table
query_create_table_1 = '''
CREATE TABLE IF NOT EXISTS music_app_history (
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    song_length double,
    PRIMARY KEY (session_id, item_in_session)
)
'''

try:
    session.execute(query_create_table_1)
except Exception as e:
    print(e)           

In [ ]:
def insert_in_table(session, tablename, **column_values):
    '''
    Executes an INSERT query for given values into a given table.

    Args:
        session: Session to execute query.
        tablename: String of table name to insert values.
        **column_values: Dictionary of columns and values to be inserted.
    '''
    query = ('INSERT INTO {}'.format(tablename)
                # list of keys to insert values for
                + ' ({}) '.format(','.join(column_values.keys()))
                # Remove the last comma
                + 'VALUES ({}) '.format(('%s,'*len(column_values))[:-1])
    )
    try:
        session.execute(query, column_values.values())
    except Exception as e:
        print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_in_table(
            session,
            tablename='music_app_history',
            session_id=int(line[8]),
            item_in_session=int(line[3]),
            artist=line[0],
            song_title=line[10],
            song_length=float(line[5])
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
### Add in the SELECT statement to verify the data was entered into the table
query_1 = '''
SELECT
    artist, song_title, song_length
FROM
    music_app_history
WHERE
    session_id = %s
    AND
    item_in_session = %s
'''

try:
    results = session.execute(query_1, (338,4))
    for r in results:
        print(r)
except Exception as e:
    print(e)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
# Drop the table if it exists
query_drop_table_2 = "DROP TABLE IF EXISTS user_session_history"
try:
    session.execute(query_drop_table_2)
except Exception as e:
    print(e)

# Create table
query_create_table_2 = '''
CREATE TABLE IF NOT EXISTS user_session_history (
    user_id int,
    session_id int,
    first_name text,
    last_name text,
    artist text,
    song_title text,
    item_in_session int,
    PRIMARY KEY ((user_id ,session_id), item_in_session)
)
'''

try:
    session.execute(query_create_table_2)
except Exception as e:
    print(e)           

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_in_table(
            session,
            tablename='user_session_history',
            user_id=int(line[10]),
            session_id=int(line[8]),
            first_name=line[1],
            last_name=line[4],
            artist=line[0],
            song_title=line[9],
            item_in_session=int(line[3])
        )

In [ ]:
query_2 = '''
SELECT
    artist, song_title, first_name, last_name
FROM
    user_session_history
WHERE
    user_id = %s
    AND
    session_id = %s
ORDER BY
    item_in_session
'''

try:
    results = session.execute(query_2, (10,182))
    for r in results:
        print(r)
except Exception as e:
    print(e)

Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
# Drop the table if it exists
query_drop_table_3 = "DROP TABLE IF EXISTS listen_history"
try:
    session.execute(query_drop_table_3)
except Exception as e:
    print(e)

# Create table
query_create_table_3 = '''
CREATE TABLE IF NOT EXISTS listen_history (
    user_id int,
    song_title text,
    first_name text,
    last_name text,
    PRIMARY KEY ((song_title), user_id)
)
'''

try:
    session.execute(query_create_table_3)
except Exception as e:
    print(e)           

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_in_table(
                    session,
                    tablename='listen_history',
                    user_id=int(line[10]),
                    song_title=line[9],
                    first_name=line[1],
                    last_name=line[4]
        )

In [ ]:
query_3 = '''
SELECT
    first_name, last_name
FROM
    listen_history
WHERE
    song_title = %s
'''

try:
    results = session.execute(query_3, ('All Hands Against His Own'))
    for r in results:
        print(r)
except Exception as e:
    print(e)

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()